In [ ]:
import sys, os
from os.path import join, dirname, abspath
import matplotlib.pyplot as plt
from matplotlib.pyplot import Figure, Axes
import numpy as np
import networkx as nx
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import make_axes_locatable
from directions import *
from string import ascii_uppercase
plt.rcParams.update({
    "text.usetex": False,
    "ytick.minor.visible":False,
    "xtick.minor.visible":False,
    'xtick.direction': "in",
    'ytick.direction': "in"
})

outdir = "out/int_1_2_desynced"
os.makedirs(outdir,exist_ok=True)
def out(fname): return join(outdir,fname)
def savefig(plot_name): 
    plt.savefig(out(plot_name),bbox_inches="tight",dpi=250)
    
import pandas as pd
from numpy.linalg import matrix_power, eig

def arr_to_latex(M):
    return '$$\n' + r'\begin{bmatrix}' + '\n' + (r'\\' + '\n').join('&'.join(str(x) for x in row) for row in M) + '\n' + r'\end{bmatrix}' + '\n' +'$$'

def vec_to_latex(x,round=3):
    return '$$\n' + r'\begin{bmatrix}' + '\n' + (r' \\ ').join(str(np.round(v,round)) for v in x) + '\n' + r'\end{bmatrix}' + '\n' +'$$'

from car import Car
from tiles import Road, Exit, Onramp
from world import World
from traffic_signals import Stoplight

In [ ]:
def plus_tile(top,right,left,bottom,cx,cy):
    road_tiles = []

    bottom_tile = bottom(4+cx,0+cy,(N+E)/2)
    road_tiles.append(bottom_tile)
    road_tiles.append(Road(4+cx,1+cy,N))
    road_tiles.append(Road(4+cx,2+cy,N))
    s1_tile = Road(4+cx,3+cy,N)
    road_tiles.append(s1_tile)
    center = Road(4+cx,4+cy,(N+E)/2)
    road_tiles.append(center)
    road_tiles.append(Road(4+cx,5+cy,N))
    road_tiles.append(Road(4+cx,6+cy,N))
    road_tiles.append(Road(4+cx,7+cy,N))
    
    top_tile = top(4+cx,8+cy,(N+E)/2)
    road_tiles.append(top_tile)

    left_tile = left(0+cx,4+cy,(N+E)/2)
    road_tiles.append(left_tile)
    road_tiles.append(Road(1+cx,4+cy,E))
    road_tiles.append(Road(2+cx,4+cy,E))
    s2_tile = Road(3+cx,4+cy,E)
    road_tiles.append(s2_tile)
    road_tiles.append(Road(5+cx,4+cy,E))
    road_tiles.append(Road(6+cx,4+cy,E))
    road_tiles.append(Road(7+cx,4+cy,E))
    right_tile = right(8+cx,4+cy,(N+E)/2)
    road_tiles.append(right_tile)
    
    return road_tiles, [top_tile,right_tile,left_tile,bottom_tile]

In [ ]:
p1,[e1t,_,o1b,o1l] = plus_tile(Exit,Road,Onramp,Onramp,0,0)
p2,[e2t,e2r,_,o2l] = plus_tile(Exit,Exit,Road,Onramp,9,0)

In [ ]:
w=World(p1+p2,cars=[])
w.draw(markersize=10)

In [ ]:
for cx,cy in zip([0,9],[0,0]):
    s = Stoplight(tile=w.tiles[4+cx,3+cy],direction=4,offset=cx,period=20)
    s2 = Stoplight(tile=w.tiles[3+cx,4+cy],direction=6,offset=10+cx,period=20)
    w.add_stoplight(s)
    w.add_stoplight(s2)

In [ ]:
w.draw(markersize=10)

In [ ]:
w.run(200,draw=True,outpath=out("intersection.gif"),markersize=10)

In [ ]:
fig, ax = plt.subplots()
cumulative = np.zeros((w.max_y+1,w.max_x+1))

c = np.zeros(len(w._tiles))
# c[onramp.index] = 1
# mkov_locs = [np.array(c, copy=True)]

def update(frame):
    global cumulative
    ax.cla()
    car_loc = w.car_info_packets[frame]
    car_map = np.zeros((w.max_y+1,w.max_x+1))
    car_locs = np.zeros((w.max_y+1,w.max_x+1))
    car_map[car_loc[:,2],car_loc[:,1]] = car_loc[:,0]
    car_locs[car_loc[:,2],car_loc[:,1]] = 1 
    cumulative += car_locs
    # mkov_locs.append(w.markov.T@mkov_locs[-1])
    ax.imshow(car_map,origin="lower",cmap="inferno")
ani = animation.FuncAnimation(fig,update,frames=len(w.car_info_packets),interval=100)
ani.save(filename=out("intersection_car_heatmap.gif"), writer="pillow",dpi=150)

In [ ]:
tiles_of_interest = np.array([[5,4],[6,4],[7,4],[8,4],[9,4],[10,4],[11,4],[12,4]])
w.tiles[tiles_of_interest]

In [ ]:
nums = []
ts = []
for t, packet in enumerate(w.car_info_packets):
    car_locs = np.zeros((w.max_y+1,w.max_x+1))
    car_locs[packet[:,2],packet[:,1]] = 1
    num_cars = np.sum(car_locs[tiles_of_interest[:,1],tiles_of_interest[:,0]])
    nums.append(num_cars)
    ts.append(t)

In [ ]:
plt.plot(ts,nums)
plt.title("Traffic in Intermediate Stretch")
plt.xlabel("t")
plt.ylabel("Cars in stretch")
ylim=plt.ylim()
going = True
s = w.signaller.stoplights[0]

gos = np.array([0]+s.gos)
stops = np.array(s.stops)

if max(s.stops) > max(s.gos):
    gos = np.concatenate([gos,[w.timestep]])
else:
    stops = np.concatenate([stops,[w.timestep]])

c = np.empty((len(gos)+len(stops),))
c[0::2] = gos
c[1::2] = stops
for i in range(len(c)-1):
    color="tab:green" if going else "tab:red"
    print(c[i],c[i+1],color)
    plt.fill_between(x=[c[i],c[i+1]],y1=[ylim[0],ylim[0]],y2=[ylim[1],ylim[1]],color=color,alpha=0.15)
    going = not going
savefig("traffic_in_stretch.png")

In [ ]:
w.signaller.stoplights

In [ ]:
going = True
s = w.signaller.stoplights[0]
c = np.empty((len(s.gos)+ len(s.stops)-1,))
c[0::2] = sorted(s.gos)
c[1::2] = sorted(s.stops[1:])
for i in range(len(c)-1):
    color="tab:green" if going else "tab:red"
    print(c[i],c[i+1],color)
    plt.fill_between(x=[c[i],c[i+1]],y1=[0,0],y2=[1,1],color=color,alpha=0.5)
    going = not going
# fill=False
# for i in range(len(s.gos)-1):
#     if fill:
#         plt.fill_between(x=[s.gos[i],s.gos[i+1]],y1=[0,0],y2=[1,1],color="tab:green",alpha=0.5)
#     fill = not fill

In [ ]:
print(c)

In [ ]:
t= np.arange(50)
lines = {}
for s in w.signaller.stoplights:
    points_south = s.direction==4
    f = np.vectorize(s.update_func)
    l = plt.plot(t,f(t),c="tab:blue" if points_south else "tab:orange",linestyle="dashed" if points_south else "dotted")
    lines[s.direction] = l[0]
plt.yticks([0,1],["Stop","Go"])
plt.xlabel("t")
plt.title("Stoplight States")
plt.legend(handles=[lines[4],lines[6]],labels=["South","East"])
savefig("signal_states.png")

In [ ]:
|